# Compressing and Cleaning Datasets

## Song data from Genius

Compressed the genius song data into only english songs, a range from 1980 to 2019, and views greater than or equal to 10000 and cleaned the lyrical column

In [3]:
!pip install pandas numpy matplotlib seaborn scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.0 MB 13.1 MB/s eta 0:00:01
   -------------------- ------------------- 5.5/11.0 MB 13.6 MB/s eta 0:00:01
   ----------------------------- ---------- 8.1/11.0 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 13.3 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 11.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   -------- ------------------------------- 2.6/12.8 MB 12.0 MB/s eta 0:00:01
   ----------------- ---------------------- 5.5/12.8 MB 12.9 MB/s eta 0:00:01
   -------------------------- ------------- 8.4/12.8 MB 13.2 MB/s eta 0:00:01
   ----------------------------------- ---- 11.3/12.8 MB 13.2 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 12.5 MB/s eta 0:00:0


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd

In [ ]:
cols = ['title', 'tag', 'artist', 'year', 'views','lyrics', 'language_cld3']


songs_df = pd.read_csv('../data/raw/song_lyrics.csv', usecols=cols, chunksize=100000)

filtered = []

for data in songs_df:
    data = data[(data['language_cld3'] == 'en') & ((data['year'] >= 1980) & (data['year'] <= 2019)) & (data['views'] >= 10000)]

    data['lyrics'] = data['lyrics'].fillna('').astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
    
    filtered.append(data)

df = pd.concat(filtered, ignore_index=True)

songs_compressed = df.groupby('year').apply(lambda x: x.sample(n=min(len(x), 275), random_state=42, replace=False)).reset_index(drop=True)

songs_compressed.to_csv('../data/processed/songs_compressed_2019.csv', index=False)

## BillBoard Hot100 Dataset (1946-2022)

1. Get rid of unwanted columns:

In [24]:
bbh100_columns_to_keep = ["Artist(s) Genres", "Hot100 Ranking Year", "Hot100 Rank"]
billboard_df = pd.read_csv('../data/raw/Billboard_Hot100_Songs_Spotify_1946-2022.csv', usecols=bbh100_columns_to_keep)

# the year filter
billboard_df = billboard_df[billboard_df["Hot100 Ranking Year"] >= 1960]

billboard_df.to_csv('../data/compressed/new_billboard_compressed.csv', index=False)

In [25]:
billboard_df.head(10)

,Artist(s) Genres,Hot100 Ranking Year,Hot100 Rank
586,"['easy listening', ""man's orchestra""]",1960,1
587,['nashville sound'],1960,2
588,"['folk rock', 'rock-and-roll', 'adult standard...",1960,3
589,"['rockabilly', 'doo-wop', 'rock-and-roll']",1960,4
590,['deep adult standards'],1960,5
591,['karaoke'],1960,6
592,"['rockabilly', 'rock-and-roll']",1960,7
593,"['doo-wop', 'rhythm and blues']",1960,8
594,"['rockabilly', 'rock-and-roll']",1960,9
595,"['rockabilly', 'rock-and-roll']",1960,10
